<a href="https://colab.research.google.com/github/veronhii/datasets/blob/main/COS30019_2023_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setup Libraries / Environment /Dependencies**

In [ ]:
!pip install scikit-plot

In [ ]:
import os
import random
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scikitplot as skplt
import sklearn

from tensorflow import keras
from shutil import copyfile
from PIL import Image
from google.colab import files
from math import sqrt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.utils import check_random_state
from sklearn import metrics
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve, ConfusionMatrixDisplay, roc_curve, auc


from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50, VGG16
from keras.optimizers import Adam
from keras.metrics import accuracy

# **Data Preparation**


## **Load Dataset**

In [ ]:
# Github Repository with its directory in Google Colab
repo_url = "https://github.com/veronhii/datasets"
repo_dir = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))

In [ ]:
# Change directory to /content
%cd /content

# Clean up the repo
!rm -rf {repo_dir}

# Git clone the repo
!git clone {repo_url}

# change directory to /content/datasets
# Check repo is up-dated
%cd {repo_dir}
!git pull

In [ ]:
# Check repo is up-dated
%cd {repo_dir}
!git pull

## **Directories**

In [ ]:
# Two folders for two classes of data
dataset_dir = os.path.join(repo_dir, "IntroAI")
glasses_dir = os.path.join(dataset_dir, "glasses")
non_glasses_dir = os.path.join(dataset_dir, "non_glasses")

# **Machine Learning Approach**

## **Data Pre-processing**

In [ ]:
# X: images, y: labels
X = []
y = []

# Append the images and the labels into X and y
for folder, label in [(glasses_dir, 0), (non_glasses_dir, 1)]:
    for file in os.listdir(folder):
        # Locate the file
        image_path = os.path.join(folder, file)
        # Convert into grayscale and array
        image = np.asarray(Image.open(image_path).convert('L'))
        # Reshape into 1D array
        image = image.reshape(-1)
        # Append the 
        X.append(image)
        y.append(label)

# Split X and y to train and test (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [ ]:
# Check the shape of the image
print(image.shape)

## **Classifiers**

In [ ]:
# Classifiers to be used
def RandomForest():
    rfclassifier = RandomForestClassifier(random_state=42, max_depth=2, min_samples_split=10, n_jobs=-1, n_estimators=10, class_weight="balanced")
    return rfclassifier

def DecisionTree():
    dtclassifier = DecisionTreeClassifier(random_state=42, max_features=1000, min_samples_leaf=100, min_samples_split=100, class_weight="balanced")
    return dtclassifier

In [ ]:
rfclassifier = RandomForest()
dtclassifier = DecisionTree()

## **Training**

In [ ]:
print(rfclassifier.fit(X_train, y_train))
print(dtclassifier.fit(X_train, y_train))

## **Save Weight**

In [ ]:
import joblib

# save
joblib.dump(rfclassifier, "/content/datasets/my_random_forest.joblib")
joblib.dump(dtclassifier, "/content/datasets/my_decision_tree.joblib")


## **Prediction**

In [ ]:
score_1 = rfclassifier.score(X_test, y_test)
y_pred_1 = rfclassifier.predict(X_test)

score_2 = dtclassifier.score(X_test, y_test)
y_pred_2 = dtclassifier.predict(X_test)

## **Evaluation (Scores)**

In [ ]:
# Random Forest Classifier - Score on test data
print("Classifier 1: Random Forest")
print("Score:",score_1)

# Decision Tree Classifier - Score on test data
print("\n\nClassifier 2: Decision Tree")
print("Score:",score_2)

In [ ]:
# Evaluation on the scores for Random Forest Classifier
print('Classifier 1: Random Forest')
mse = mean_squared_error(y_test, y_pred_1)
print('Mean Square Error:', mse)
rmse = sqrt(mse)
print('Root Mean Square Error:', rmse)
accuracy = accuracy_score(y_test, y_pred_1)
print('Accuracy:', accuracy)
precision = precision_score(y_test, y_pred_1)
print('Precision:', precision)
recall = recall_score(y_test, y_pred_1)
print('Recall:', recall)
f1 = f1_score(y_test, y_pred_1)
print('F1 score:', f1)

# Evaluation on the scores for Decision Trees Classifier
print('\n\nClassifier 2: Decision Trees')
mse = mean_squared_error(y_test, y_pred_2)
print('Mean Square Error:', mse)
rmse = sqrt(mse)
print('Root Mean Square Error:', rmse)
accuracy = accuracy_score(y_test, y_pred_2)
print('Accuracy:', accuracy)
precision = precision_score(y_test, y_pred_2)
print('Precision:', precision)
recall = recall_score(y_test, y_pred_2)
print('Recall:', recall)
f1 = f1_score(y_test, y_pred_2)
print('F1 score:', f1)

## **Visualisation on Graphs**

In [ ]:
# Precision-Recall-Curve (Random Forest Classifier)
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_1)
plt.plot(recall, precision)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve (Random Forest Classifier)')
plt.show()

print("\n\n")

# Precision-Recall-Curve (Decision Tree Classifier)
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_2)
plt.plot(recall, precision)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve (Decision Tree Classifier)')
plt.show()

In [ ]:
# ROC curve (Random Forest Classifier)
# Get the predicted probabilities for the test set
y_prob_1 = rfclassifier.predict_proba(X_test)[:, 1]
# Compute the false positive rate, true positive rate, and thresholds
fpr1, tpr1, thresholds = roc_curve(y_test, y_prob_1)
# Compute the area under the ROC curve
roc_auc = auc(fpr1, tpr1)

plt.plot(fpr1, tpr1, color='darkorange', label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic for Random Forest Classifier')
plt.legend(loc="lower right")
plt.show()

print("\n\n")

# ROC curve (Decision Trees Classifier)
# Get the predicted probabilities for the test set
y_prob_2 = dtclassifier.predict_proba(X_test)[:, 1]
# Compute the false positive rate, true positive rate, and thresholds
fpr2, tpr2, thresholds = roc_curve(y_test, y_prob_2)
# Compute the area under the ROC curve
roc_auc = auc(fpr2, tpr2)

plt.plot(fpr2, tpr2, color='darkorange', label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic for Decision Trees Classifier')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Confusion Matrix (Random Forest Classifier)
f,ax=plt.subplots(1,1,figsize=(8,8))
skplt.metrics.plot_confusion_matrix(y_test, y_pred_1, normalize='true',ax=ax)
plt.title('Confusion Matrix (Random Forest Classifier)')
plt.show()

print("\n\n")

# Confusion Matrix (Decision Trees Classifier)
f,ax=plt.subplots(1,1,figsize=(8,8))
skplt.metrics.plot_confusion_matrix(y_test, y_pred_2, normalize='true',ax=ax)
plt.title('Confusion Matrix (Decision Trees Classifier)')
plt.show()

# **Deep Learning Approach**

In [ ]:
train_percent = 0.7
val_percent = 0.15
test_percent = 0.15

In [ ]:
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')
test_dir = os.path.join(dataset_dir, 'test')

if not os.path.exists(train_dir):
    os.mkdir(train_dir)
if not os.path.exists(val_dir):
    os.mkdir(val_dir)
if not os.path.exists(test_dir):
    os.mkdir(test_dir)

for folders in ["train", "val", "test"]:
    glasses_dir = os.path.join(dataset_dir, folders, "glasses")
    non_glasses_dir = os.path.join(dataset_dir, folders, "non_glasses")

    if not os.path.exists(glasses_dir):
        os.mkdir(glasses_dir)
    if not os.path.exists(non_glasses_dir):
        os.mkdir(non_glasses_dir)

In [ ]:
for classes in ["glasses","non_glasses"]:
    # get a list of all the image files in the animal folder
    folder = os.path.join(dataset_dir, classes)
    files = os.listdir(folder)
    random.shuffle(files)
    
    # split the files into train, val, and test sets
    train_end = int(len(files) * train_percent)
    val_end = int(len(files) * (train_percent + val_percent))
    train_files = files[:train_end]
    val_files = files[train_end:val_end]
    test_files = files[val_end:]
    
    # copy the files into the corresponding train, val, and test folders
    for file in train_files:
        copyfile(os.path.join(folder, file), os.path.join(dataset_dir, "train", classes, file))
    for file in val_files:
        copyfile(os.path.join(folder, file), os.path.join(dataset_dir, "val", classes, file))
    for file in test_files:
        copyfile(os.path.join(folder, file), os.path.join(dataset_dir, "test", classes, file))

In [ ]:
len(os.listdir(os.path.join(dataset_dir, "train","non_glasses")))

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(224, 224),
    class_mode='binary',
    batch_size=64,
    shuffle=True
)

val_data = val_test_datagen.flow_from_directory(
    directory=val_dir,
    target_size=(224, 224),
    class_mode='binary',
    batch_size=64,
    shuffle=True
)

test_data = val_test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(224, 224),
    class_mode='binary',
    batch_size=64,
    shuffle=True
)

In [ ]:
class_indices = test_data.class_indices
print(class_indices)

In [ ]:
def resnet50():
  pretrained_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet", pooling='max')
  for layer in pretrained_model.layers[:-5]:
      layer.trainable = False
  return pretrained_model
  
def vgg16():
  pretrained_model = VGG16(input_shape=(224, 224,3), include_top=False, weights="imagenet", pooling='max')
  for layer in pretrained_model.layers[:-5]:
      layer.trainable = False
  return pretrained_model


In [ ]:
def create_model(pretrained_model):
  model = Sequential()
  model.add(pretrained_model)
  model.add(Dense(256, activation='relu'))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [ ]:
pretrained_model_resnet50 = resnet50()
pretrained_model_vgg16 = vgg16()

model1 = create_model(pretrained_model_resnet50)
model2 = create_model(pretrained_model_vgg16)

model1.summary()
print("\n\n")
model2.summary()

In [ ]:
model1.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
model2.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history1 = model1.fit(train_data,
          validation_data = val_data,
          steps_per_epoch = train_data.n//train_data.batch_size,
          validation_steps = val_data.n//val_data.batch_size,
          epochs=10)

history2 = model2.fit(train_data,
          validation_data = val_data,
          steps_per_epoch = train_data.n//train_data.batch_size,
          validation_steps = val_data.n//val_data.batch_size,
          epochs=10)

In [ ]:
# Save the model 
# Choose one name to save according to the architecture used
model1.save("Resnet50_GlassesVsNonglasses.h5")
model2.save("VGG16_GlassesVsNonglasses.h5")

In [ ]:
from google.colab import files
files.download("Resnet50_GlassesVsNonglasses.h5")
files.download("VGG16_GlassesVsNonglasses.h5")

In [ ]:
path_saved_model = "/content/datasets/"
model1_name = "Resnet50_GlassesVsNonglasses.h5"
model2_name = "VGG16_GlassesVsNonglasses.h5"

model1 = load_model(os.path.join(path_saved_model, model1_name))
model2 = load_model(os.path.join(path_saved_model, model2_name))

score = model1.evaluate(val_data)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
score = model2.evaluate(val_data)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
predict_1 = model1.predict(test_data)
predict_2 = model2.predict(test_data)

In [ ]:
y_classes_1 = np.where(predict_1 > 0.5, 1, 0)
y_classes_2 = np.where(predict_2 > 0.5, 1, 0)

In [ ]:
# Check repo is up-dated
%cd {repo_dir}
!git pull

## **Evaluation (Scores)**

In [ ]:
# Retrieve the true labels from the ImageDataGenerator
test_labels = test_data.labels

# Evaluation on the scores for VGG16
print('Model 1: ResNet50')
mse = mean_squared_error(test_labels, y_classes_1)
print('Mean Square Error:', mse)
rmse = sqrt(mse)
print('Root Mean Square Error:', rmse)
accuracy = accuracy_score(test_labels, y_classes_1)
print('Accuracy:', accuracy)
precision = precision_score(test_labels, y_classes_1, average='macro', zero_division=1)
print('Precision:', precision)
recall = recall_score(test_labels, y_classes_1, average='macro')
print('Recall:', recall)
f1 = f1_score(test_labels, y_classes_1, average='macro')
print('F1 score:', f1)

# Evaluation on the scores for VGG16
print('\n\nModel 2: VGG16')
mse = mean_squared_error(test_labels, y_classes_2)
print('Mean Square Error:', mse)
rmse = sqrt(mse)
print('Root Mean Square Error:', rmse)
accuracy = accuracy_score(test_labels, y_classes_2)
print('Accuracy:', accuracy)
precision = precision_score(test_labels, y_classes_2, average='macro', zero_division=1)
print('Precision:', precision)
recall = recall_score(test_labels, y_classes_2, average='macro')
print('Recall:', recall)
f1 = f1_score(test_labels, y_classes_2, average='macro')
print('F1 score:', f1)

## **Visualisations on Graph**

In [ ]:
# Precision-Recall-Curve (Resnet50)
precision, recall, thresholds = precision_recall_curve(test_labels, y_classes_1)
plt.plot(recall, precision)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve (Resnet50)')
plt.show()

print("\n\n")

# Precision-Recall-Curve (VGG16)
precision, recall, thresholds = precision_recall_curve(test_labels, y_classes_2)
plt.plot(recall, precision)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve (VGG16)')
plt.show()

In [ ]:
# Confusion Matrix (Resnet50)
f,ax=plt.subplots(1,1,figsize=(8,8))
skplt.metrics.plot_confusion_matrix(test_labels, y_classes_1, normalize='true',ax=ax)
plt.title('Confusion Matrix (Resnet50)')
plt.show()

print("\n\n")

# Confusion Matrix (VGG16)
f,ax=plt.subplots(1,1,figsize=(8,8))
skplt.metrics.plot_confusion_matrix(test_labels, y_classes_2, normalize='true',ax=ax)
plt.title('Confusion Matrix (VGG16)')
plt.show()

# **LIVE CAMERA FEED FACIAL DETECTION**

In [ ]:
import numpy as np
import os
import cv2
from tensorflow import keras
from keras.models import load_model 
from keras.utils import img_to_array

import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
# Helper libraries
import numpy as np
import matplotlib.pyplot as pl
import joblib


path_saved_model = '/content/datasets/'
model1_name = 'Resnet50_GlassesVsNonglasses.h5'
model2_name = 'VGG16_GlassesVsNonglasses.h5'

model1 = load_model(os.path.join(path_saved_model, model1_name))
model2 = load_model(os.path.join(path_saved_model, model2_name))


loaded_rf = RandomForestClassifier()
loaded_dt = DecisionTreeClassifier()
# # load
loaded_rf = joblib.load("/content/datasets/my_random_forest.joblib")
loaded_dt = joblib.load("/content/datasets/my_decision_tree.joblib")



In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes
     
     
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "Status:";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '' +
          'When finished, click here or on the video to stop this demo';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
from PIL import Image, ImageDraw, ImageFont
from google.colab.patches import cv2_imshow
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 
while True:


    # ========================================machine learning

    # js_reply = video_frame(label_html, bbox)
    # if not js_reply:
    #     break

    # # convert JS response to OpenCV Image
    # img = js_to_image(js_reply["img"])

    # # Convert into grayscale
    # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # resized_img = cv2.resize(gray, (1024, 1024))
      
    # # for machine learning approach
    # resized_img = resized_img.reshape(-1)

    # prediction = loaded_rf.predict([resized_img])
    # # prediction = model2.predict(img_preprocessed, verbose=0)

    # if(prediction == 0):
    #   predicted_class = "Glasses"
    # else:
    #   predicted_class = "Non Glasses"

    
    # bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # # creating a image object 
    # img = Image.fromarray(bbox_array)
    # draw = ImageDraw.Draw(img) 
      
    # # specified font size
    # font = ImageFont.truetype('/content/datasets/arial.ttf', 50) 
      
    # text = predicted_class
      
    # text_color = (255, 0, 0)

    # # drawing text size
    # draw.text((5, 5), text, font = font, fill=text_color, align ="left") 

    # bbox = np.array(img) 
    # # convert overlay of bbox into bytes
    # bbox_bytes = bbox_to_bytes(bbox)
    # # update bbox so the next frame gets a new overlay
    # bbox = bbox_bytes



    # ========================================deep learning

    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])

    resized_img = cv2.resize(img, (224, 224))
    # Image.fromarray(resized_img).show()
    img_array =image.img_to_array(resized_img)
    img_batch = np.expand_dims(img_array, axis=0)
    img_preprocessed = preprocess_input(img_batch)
    prediction = model1.predict(img_preprocessed, verbose=0)

    if(prediction[0][0] > 0.5):
      predicted_class = "Glasses"
    else:
      predicted_class = "Non Glasses"

    
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # creating a image object 
    img = Image.fromarray(bbox_array)
    draw = ImageDraw.Draw(img) 
      
    # specified font size
    font = ImageFont.truetype('/content/datasets/arial.ttf', 50) 
      
    text = predicted_class + str(prediction[0][0])
      
    text_color = (255, 0, 0)

    # drawing text size
    draw.text((5, 5), text, font = font, fill=text_color, align ="left") 

    bbox = np.array(img) 
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox)
    # update bbox so the next frame gets a new overlay
    bbox = bbox_bytes
    
